This Notebook is for customer churn data from IDM. 

In [ ]:
pip install kaggle 


   -------- ------------------------------- 1/5 [tqdm]
   -------- ------------------------------- 1/5 [tqdm]
   -------- ------------------------------- 1/5 [tqdm]
   ---------------- ----------------------- 2/5 [python-slugify]
   ------------------------ --------------- 3/5 [protobuf]
   ------------------------ --------------- 3/5 [protobuf]
   ------------------------ --------------- 3/5 [protobuf]
   ------------------------ --------------- 3/5 [protobuf]
   ------------------------ --------------- 3/5 [protobuf]
   ------------------------ --------------- 3/5 [protobuf]
   ------------------------ --------------- 3/5 [protobuf]
   -------------------------------- ------- 4/5 [kaggle]
   -------------------------------- ------- 4/5 [kaggle]
   -------------------------------- ------- 4/5 [kaggle]
   -------------------------------- ------- 4/5 [kaggle]
   -------------------------------- ------- 4/5 [kaggle]
   -------------------------------- ------- 4/5 [kaggle]
   -----------


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
blastchar_telco_customer_churn_path = kagglehub.dataset_download('blastchar/telco-customer-churn')

print('Data source import complete.')
# You can now access the data at the path:


ModuleNotFoundError: No module named 'kagglehub'